# Working with PySpark DataFrames and exploring the data

In [1]:
# Change width of Jupyter notebook
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .appName("comm") \
        .getOrCreate()

# Read in the dataset and show the number of records

In [3]:
# note that read.json can read a zipped JSON directly
# df = spark.read.json('part-00000-a159c41a-bc58-4476-9b78-c437667f9c2b-c000.json.gz')

df = spark.read.csv("NYC_Merged_Complaints_Data.csv",inferSchema=True, header=True)

In [4]:
df.count()
##2834417 when you don't have header=True

276401

# Print the schema

In [5]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- ComplaintID: integer (nullable = true)
 |-- ProblemID: integer (nullable = true)
 |-- UnitTypeID: integer (nullable = true)
 |-- UnitType: string (nullable = true)
 |-- SpaceTypeID : integer (nullable = true)
 |-- SpaceType: string (nullable = true)
 |-- TypeID: integer (nullable = true)
 |-- Type: string (nullable = true)
 |-- MajorCategoryID: integer (nullable = true)
 |-- MajorCategory: string (nullable = true)
 |-- MinorCategoryID: integer (nullable = true)
 |-- MinorCategory: string (nullable = true)
 |-- CodeID: integer (nullable = true)
 |-- Code: string (nullable = true)
 |-- StatusID: integer (nullable = true)
 |-- Status: string (nullable = true)
 |-- StatusDate: string (nullable = true)
 |-- StatusDescription: string (nullable = true)
 |-- BuildingID: integer (nullable = true)
 |-- BoroughID: integer (nullable = true)
 |-- Borough: string (nullable = true)
 |-- HouseNumber: string (nullable = true)
 |-- StreetName: string (nullab

# Show the first 5 records

In [6]:
# df.select('*').show(5)
df.show(5)

+---+-----------+---------+----------+------------+------------+----------------+------+---------+---------------+-------------+---------------+-------------+------+--------------------+--------+------+----------+--------------------+----------+---------+-------+-----------+----------+-------+-----+---+---------+--------------+------------+
|_c0|ComplaintID|ProblemID|UnitTypeID|    UnitType|SpaceTypeID |       SpaceType|TypeID|     Type|MajorCategoryID|MajorCategory|MinorCategoryID|MinorCategory|CodeID|                Code|StatusID|Status|StatusDate|   StatusDescription|BuildingID|BoroughID|Borough|HouseNumber|StreetName|    Zip|Block|Lot|Apartment|CommunityBoard|ReceivedDate|
+---+-----------+---------+----------+------------+------------+----------------+------+---------+---------------+-------------+---------------+-------------+------+--------------------+--------+------+----------+--------------------+----------+---------+-------+-----------+----------+-------+-----+---+---------+

# Location
Count the number of records where the zipcode is 11216 (where I live)

In [7]:
# sql, make dataframe accessible
df.createOrReplaceTempView('zip')

# this selects the 'Zip' column from the dataframe and saves it to the 'sqlDF' variable
zipc = spark.sql("SELECT * FROM zip") #.show(1)

# .select() goes into the 'address' column (the only column in this reduced dataframe) and then selects the city from each row of this column
zipcode = zipc.select('zip.Zip') #.show()

# now that we have the cities isolated, we can return only the rows of the dataframe where the city is 'Houston'
#houston = spark.sql("SELECT * FROM addr where address.city like '%Houston%'") #.show()

# also we need to use <<== 'Houston'>> instead of <<like '%[]%'>> to account for "South Houston"
bedstuy = spark.sql("SELECT * FROM zip where zip == '11216'") #.show()
#houston.show()
bedstuy.count()

3793

Count the number of records in West Village (10014) ... should be a lot less 

In [8]:
# sql, make dataframe accessible
df.createOrReplaceTempView('zip')

# this selects the 'Zip' column from the dataframe and saves it to the 'sqlDF' variable
zipc = spark.sql("SELECT * FROM zip") #.show(1)

# .select() goes into the 'address' column (the only column in this reduced dataframe) and then selects the city from each row of this column
zipcode = zipc.select('zip.Zip') #.show()

# now that we have the cities isolated, we can return only the rows of the dataframe where the city is 'Houston'
#houston = spark.sql("SELECT * FROM addr where address.city like '%Houston%'") #.show()

# also we need to use <<== 'Houston'>> instead of <<like '%[]%'>> to account for "South Houston"
westvillage = spark.sql("SELECT * FROM zip where zip == '10014'") #.show()
#houston.show()
westvillage.count()

772

# Calculate close time for each complaint

In [9]:
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.feature import HashingTF
from pyspark.mllib.classification import LogisticRegressionWithSGD
from pyspark.sql import SparkSession
from pyspark.sql.functions import pandas_udf, PandasUDFType, col, split, explode
from pyspark.sql.types import *
import pandas as pd

from pyspark.sql.functions import unix_timestamp

# preprocessing: convert dates to datetime format and calculate response variable - closing time
timeDiff = (unix_timestamp('StatusDate', 'MM/dd/yyyy') - unix_timestamp('ReceivedDate', 'MM/dd/yyyy')) / 86400 #seconds per day
timeDiff = timeDiff.cast(IntegerType())

# add closing time (time for complaint to be resolved) to dataframe
merged_df = df.withColumn("closeTime", timeDiff)
merged_df.show(5)
#Reference: https://stackoverflow.com/questions/47701339/subtracting-two-date-columns-in-pyspark-python

+---+-----------+---------+----------+------------+------------+----------------+------+---------+---------------+-------------+---------------+-------------+------+--------------------+--------+------+----------+--------------------+----------+---------+-------+-----------+----------+-------+-----+---+---------+--------------+------------+---------+
|_c0|ComplaintID|ProblemID|UnitTypeID|    UnitType|SpaceTypeID |       SpaceType|TypeID|     Type|MajorCategoryID|MajorCategory|MinorCategoryID|MinorCategory|CodeID|                Code|StatusID|Status|StatusDate|   StatusDescription|BuildingID|BoroughID|Borough|HouseNumber|StreetName|    Zip|Block|Lot|Apartment|CommunityBoard|ReceivedDate|closeTime|
+---+-----------+---------+----------+------------+------------+----------------+------+---------+---------------+-------------+---------------+-------------+------+--------------------+--------+------+----------+--------------------+----------+---------+-------+-----------+----------+-------+

In [10]:
# Now reduce it
reduced_df = merged_df[['ComplaintID', 'ProblemID', 'SpaceType', 'TypeID', 'Type', 'MajorCategoryID', 'MajorCategory', 'MinorCategoryID', 'MinorCategory', 'CodeID', 'Code', 'Zip', 'StatusDate', 'ReceivedDate', 'closeTime']]

reduced_df.show(8)

+-----------+---------+----------------+------+-------------+---------------+-------------+---------------+-------------+------+--------------------+-------+----------+------------+---------+
|ComplaintID|ProblemID|       SpaceType|TypeID|         Type|MajorCategoryID|MajorCategory|MinorCategoryID|MinorCategory|CodeID|                Code|    Zip|StatusDate|ReceivedDate|closeTime|
+-----------+---------+----------------+------+-------------+---------------+-------------+---------------+-------------+------+--------------------+-------+----------+------------+---------+
|    2397487|  3768602|ENTIRE APARTMENT|     2|    HAZARDOUS|             13|     NONCONST|            106|       VERMIN|   886|             ROACHES|11432.0|08/12/2004|  06/20/2004|       53|
|    2397487|  3768603|           OTHER|     2|    HAZARDOUS|             13|     NONCONST|            106|       VERMIN|   884|                MICE|11432.0|08/12/2004|  06/20/2004|       53|
|    2397487|  3768604| ENTIRE BUILDING|

# Export

In [11]:
reduced_df.coalesce(1).write.option("header","true").option("sep",",").mode("overwrite").csv("output/path")

In [12]:
# sql, make dataframe accessible
reduced_df.createOrReplaceTempView('addr')

# updating the select to return everything and specifying the where clause to look for cases that took longer than 60 days to close 
greater30 = spark.sql("SELECT * FROM addr where closeTime >= '60'")

greater30.count()

10545

### Note: 
That is pretty insane. Over 100k complaints took longer than 2 months to close.

# Location and closeTime
Does it take longer to close in poor neighborhoods vs. in rich neighborhoods?

In [13]:
# West Village
spark.sql("SELECT * FROM addr where Zip == '10014' and closeTime >= '60'").count() #.show() 

38

In [14]:
# SoHo
spark.sql("SELECT * FROM addr where Zip == '10013' and closeTime >= '60'").count() #.show()

37

In [15]:
# Chelsea
spark.sql("SELECT * FROM addr where Zip == '10001' and closeTime >= '60'").count() #.show()

25

In [16]:
# TriBeCa
spark.sql("SELECT * FROM addr where Zip == '10007' and closeTime >= '60'").count() #.show()

7

In [17]:
# Bedford-Stuyvesant
spark.sql("SELECT * FROM addr where Zip == '11216' and closeTime >= '60'").count() #.show()

116

In [18]:
# Bronx, Morris Heights
spark.sql("SELECT * FROM addr where Zip == '10453' and closeTime >= '60'").count() #.show()

357

In [19]:
# Bronx, Melrose
spark.sql("SELECT * FROM addr where Zip == '10456' and closeTime >= '60'").count() #.show()

270

In [20]:
# Bronx, Bathgate
spark.sql("SELECT * FROM addr where Zip == '10457' and closeTime >= '60'").count() #.show()

223

# Income
It would be really great if we could pull in zip code level (household) income data and then show the relationship between income and time it takes for these complaints to be closed. After this cursory search by zipcodes, I think that the data we have will illustrate how low-income tenants suffer disproportionatley to high-income tenants. 